## OMOP CDM to Beacon v2 Models
Google Colab to perform the conversion from a PosgreSQL export to the _individuals_ entity of the [Beacon v2 Models](https://docs.genomebeacons.org/schemas-md/individuals_defaultSchema)


# Installation of the software `Convert-Pheno`

*Note:* We'll perform an installation directly from the [GitHub](https://github.com/mrueda/convert-pheno) source.

First we will download the repository by using `git clone`

In [ ]:
!git clone https://github.com/CNAG-Biomedical-Informatics/convert-pheno.git


Cloning into 'convert-pheno'...
remote: Enumerating objects: 5960, done.
remote: Counting objects: 100% (1325/1325), done.
remote: Compressing objects: 100% (438/438), done.
remote: Total 5960 (delta 718), reused 1292 (delta 696), pack-reused 4635
Receiving objects: 100% (5960/5960), 81.77 MiB | 17.40 MiB/s, done.
Resolving deltas: 100% (3441/3441), done.
Updating files: 100% (157/157), done.


Now we enter into `convert-pheno` directory:

In [ ]:
%cd convert-pheno/

/content/convert-pheno


And we perform the installation of the needed dependencies.
Note that this task will take **a few minutes** to complete...

In [ ]:
!make -f makefile.install install

sudo apt-get install cpanminus libbz2-dev zlib1g-dev libperl-dev libssl-dev
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
libbz2-dev is already the newest version (1.0.8-5build1).
libbz2-dev set to manually installed.
libssl-dev is already the newest version (3.0.2-0ubuntu1.15).
zlib1g-dev is already the newest version (1:1.2.11.dfsg-2ubuntu9.2).
zlib1g-dev set to manually installed.
The following additional packages will be installed:
  libalgorithm-c3-perl libany-uri-escape-perl libb-hooks-endofscope-perl
  libb-hooks-op-check-perl libclass-c3-perl libclass-c3-xs-perl
  libclass-method-modifiers-perl libclass-xsaccessor-perl libclone-perl
  libcpan-changes-perl libcpan-distnameinfo-perl libcpan-meta-check-perl
  libdata-optlist-perl libdata-perl-perl libdata-section-perl
  libdevel-callchecker-perl libdynaloader-functions-perl libencode-locale-perl
  libexporter-tiny-perl libfile-pushd-perl libfile-slurp-perl
  libgetopt-long-descrip

Ok, finally let's run some tests to make sure all works as it should:

In [ ]:
!make -f makefile.install test

prove -l
t/args.t ..... ok
t/cli.t ...... ok
t/mapping.t .. ok
t/module.t ... ok
t/stream.t ... ok
All tests successful.
Files=5, Tests=33, 38 wallclock secs ( 0.04 usr  0.01 sys + 36.11 cusr  1.01 csys = 37.17 CPU)
Result: PASS


The CLI is located at `bin/convert-pheno`. Let's execute it to visualize the help:

In [ ]:
!bin/convert-pheno -h

Usage:
     convert-pheno [-i input-type] <infile> [-o output-type] <outfile> [-options]

         Arguments:                       
           (input-type): 
                 -ibff                    Beacon v2 Models ('individuals' JSON|YAML) file
                 -iomop                   OMOP-CDM CSV files or PostgreSQL dump
                 -ipxf                    Phenopacket v2 (JSON|YAML) file
                 -iredcap (experimental)  REDCap (raw data) export CSV file
                 -icdisc  (experimental)  CDISC-ODM v1 XML file
                 -icsv    (experimental)  Raw data CSV

                 (Wish-list)
                 #-iopenehr               openEHR
                 #-ifhir                  HL7/FHIR

           (output-type):
                 -obff                    Beacon v2 Models ('individuals' JSON|YAML) file
                 -opxf                    Phenopacket v2 (JSON|YAML) file

                 (Wish-list)
                 #-oomop                  OMOP-CDM

# Converting OMOP CDM data to Beacon v2

For the explanation we will be using the synthtic data [EUNOMIA](https://ohdsi.github.io/Eunomia). These data were imported into a PostgreSQL instance and then exported as a single [text file](/content/convert-pheno/t/omop2bff/in/omop_cdm_eunomia.sql).

Note that `Convert-Pheno` also allows to perform the same operations using separated CSV files (one for each table) as input.

## OMOP CDM to BFF

Here we'll convert the OMOP CDM export to Beacon Friendly Format (BFF). BFF is the data exchange format (`JSON`) of the Beacon v2 Models.

From now on. we'll be storing the files in the directory `/content`.

### No-stream mode (default)

In [ ]:
!bin/convert-pheno -iomop t/omop2bff/in/omop_cdm_eunomia.sql -obff omop_individuals.json --out-dir /content

Uhmm...no output on the screen, what happened?

Well, this is actually a sign that the program worked fine. No news is good news.

A new file was created at `/content/omop_individuals.json`.

*Note:* You can access the directory `/content` by clicking in the folder icon on the left side navigation menu.

If you want to re-create a file and the script says the file exists you can use the option `-O` (overwrite).

Let's use the Linux command `head` see the first 10 lines of the file.

In [ ]:
!head /content/omop_individuals.json

[
{
   "diseases" : [
      {
         "_info" : {
            "CONDITION_OCCURRENCE" : {
               "OMOP_columns" : {
                  "condition_concept_id" : 40479768,
                  "condition_end_date" : "1981-09-16",
                  "condition_end_datetime" : "1981-09-16 00:00:00",


Browsing JSON text files can be a bit cumbersome. We will install the awesome utility [jq](https://stedolan.github.io/jq) to extract some information.

In [ ]:
!apt install jq

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libjq1 libonig5
The following NEW packages will be installed:
  jq libjq1 libonig5
0 upgraded, 3 newly installed, 0 to remove and 41 not upgraded.
Need to get 357 kB of archives.
After this operation, 1,087 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/main amd64 libonig5 amd64 6.9.7.1-2build1 [172 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/main amd64 libjq1 amd64 1.6-2.1ubuntu3 [133 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/main amd64 jq amd64 1.6-2.1ubuntu3 [52.5 kB]
Fetched 357 kB in 1s (549 kB/s)
Selecting previously unselected package libonig5:amd64.
(Reading database ... 123066 files and directories currently installed.)
Preparing to unpack .../libonig5_6.9.7.1-2build1_amd64.deb ...
Unpacking libonig5:amd64 (6.9.7.1-2build1) ...
Selecting previously unselected package libjq1:am

Great! The created JSON file consists of an array of objects, each object being an invdividual. Now we we'll check the property "id" for all individuals, but we will only display the first 5.

In [ ]:
!jq '.[].id' /content/omop_individuals.json | head -5

"1"
"2"
"3"
"5"
"6"


Great!

Ok, let's use `jq` again to fetch the `sex` property from each individual and count how many we have from each.

In [ ]:
!jq '.[].sex.label' /content/omop_individuals.json | sort | uniq -c

    256 "Female"
    244 "Male"


Let's complicate a bit the query. Let's find out the disease ontology-term id for the 1st item inside `diseases`.

In [ ]:
!jq '.[].diseases[0].diseaseCode.id' /content/omop_individuals.json | sort | uniq -c | sort -nr -k1

    422 null
     24 "SNOMED:444814009"
     11 "SNOMED:195662009"
      8 "SNOMED:10509002"
      4 "SNOMED:43878008"
      3 "SNOMED:62106007"
      3 "SNOMED:40055000"
      2 "SNOMED:74474003"
      2 "SNOMED:70704007"
      2 "SNOMED:65363002"
      2 "SNOMED:444470001"
      2 "SNOMED:39848009"
      2 "SNOMED:396275006"
      2 "SNOMED:301011002"
      1 "SNOMED:75498004"
      1 "SNOMED:410429000"
      1 "SNOMED:403191005"
      1 "SNOMED:370247008"
      1 "SNOMED:36971009"
      1 "SNOMED:284549007"
      1 "SNOMED:283385000"
      1 "SNOMED:26929004"
      1 "SNOMED:263102004"
      1 "SNOMED:16114001"
      1 "SNOMED:128613002"


Ok. SO we see we have a bunch of `null`values, and then 2-3 predominant conditions.

Let's now use the flag `--verbose` to get more information on the job itself:

In [ ]:
!bin/convert-pheno -iomop t/omop2bff/in/omop_cdm_eunomia.sql -obff omop_individuals.json --out-dir /content --verbose -O

****************************************
*  Phenotypic Data Model Convert Tool  *
*          - CONVERT-PHENO -           *
*            Version: 0.20_1             *
*   (C) 2022-2024 Manuel Rueda, CNAG   *
*       The Artistic License 2.0       *
****************************************

==== METHOD: <omop2bff> ====
Processing: <t/omop2bff/in/omop_cdm_eunomia.sql>
Writing </content/omop_individuals.json> file

Processing table ... <CONCEPT>
Rows total:     444

Processing table ... <CONDITION_OCCURRENCE>
Rows total:     500

Processing table ... <DRUG_EXPOSURE>
Rows total:     500

Processing table ... <MEASUREMENT>
Rows total:     500

Processing table ... <OBSERVATION>
Rows total:     500

Processing table ... <PERSON>
Rows total:     500

Processing table ... <PROCEDURE_OCCURRENCE>
Rows total:     500

Processing table ... <VISIT_OCCURRENCE>
Rows total:     500

Individuals total:     500



Let's take a look to the data with `pandas`. We'll be displaying only the 1st individual.

In [ ]:
import pandas as pd
from google.colab import data_table
data_table.enable_dataframe_formatter()

# Import json
df = pd.read_json('/content/omop_individuals.json')
df.head(1)

,diseases,ethnicity,geographicOrigin,id,info,interventionsOrProcedures,sex,phenotypicFeatures,treatments
0,[{'_info': {'CONDITION_OCCURRENCE': {'OMOP_col...,"{'id': 'NCIT:C41261', 'label': 'White'}","{'id': 'NCIT:C43857', 'label': 'Italian'}",1,{'PERSON': {'OMOP_columns': {'birth_datetime':...,[{'_info': {'PROCEDURE_OCCURRENCE': {'OMOP_col...,"{'id': 'NCIT:C20197', 'label': 'Male'}",NaN,NaN


Now we'll run `Convert-Pheno` again but we'll ask `Convert-Pheno` to print the SQL tables as `csv` files.

We'll be using the flag:

* `--sql2csv`

In [ ]:
!bin/convert-pheno -iomop t/omop2bff/in/omop_cdm_eunomia.sql -obff omop_individuals.json --out-dir /content --sql2csv --verbose -O

****************************************
*  Phenotypic Data Model Convert Tool  *
*          - CONVERT-PHENO -           *
*            Version: 0.20_1             *
*   (C) 2022-2024 Manuel Rueda, CNAG   *
*       The Artistic License 2.0       *
****************************************

==== METHOD: <omop2bff> ====
Processing: <t/omop2bff/in/omop_cdm_eunomia.sql>
Writing </content/omop_individuals.json> file

Processing table ... <CONCEPT>
Rows total:     444

Processing table ... <CONDITION_OCCURRENCE>
Rows total:     500

Processing table ... <DRUG_EXPOSURE>
Rows total:     500

Processing table ... <MEASUREMENT>
Rows total:     500

Processing table ... <OBSERVATION>
Rows total:     500

Processing table ... <PERSON>
Rows total:     500

Processing table ... <PROCEDURE_OCCURRENCE>
Rows total:     500

Processing table ... <VISIT_OCCURRENCE>
Rows total:     500

Individuals total:     500



And voila...now you have a bunch of `csv` files inside `/content`

The tables `PERSON` and `CONCEPT` are always read, but it is possible to process specific tables by using the option `--omop-tables`. Let's process only the table `DRUG_EXPOSURE`:


In [ ]:
!bin/convert-pheno -iomop t/omop2bff/in/omop_cdm_eunomia.sql -obff omop_individuals_drug_exposure.json --omop-tables DRUG_EXPOSURE --out-dir /content --verbose

****************************************
*  Phenotypic Data Model Convert Tool  *
*          - CONVERT-PHENO -           *
*            Version: 0.20_1             *
*   (C) 2022-2024 Manuel Rueda, CNAG   *
*       The Artistic License 2.0       *
****************************************

==== METHOD: <omop2bff> ====
Processing: <t/omop2bff/in/omop_cdm_eunomia.sql>
Writing </content/omop_individuals_drug_exposure.json> file

Processing table ... <CONCEPT>
Rows total:     444

Processing table ... <DRUG_EXPOSURE>
Rows total:     500

Processing table ... <PERSON>
Rows total:     500

Individuals total:     500



Finally, you may have noticed that we only had 500 individuals in our examples. This is deliberate. We set a limit to avoid people crashing their machines while testing the code. If you want to convert your whole OMOP CDM export, use the option `-max-lines-sql` and set it to, say, 99999, like this:

In [ ]:
!bin/convert-pheno -iomop t/omop2bff/in/omop_cdm_eunomia.sql -obff omop_individuals_drug_exposure.json --omop-tables DRUG_EXPOSURE -O --out-dir /content --verbose -max-lines-sql 99999

****************************************
*  Phenotypic Data Model Convert Tool  *
*          - CONVERT-PHENO -           *
*            Version: 0.20_1             *
*   (C) 2022-2024 Manuel Rueda, CNAG   *
*       The Artistic License 2.0       *
****************************************

==== METHOD: <omop2bff> ====
Processing: <t/omop2bff/in/omop_cdm_eunomia.sql>
Writing </content/omop_individuals_drug_exposure.json> file

Processing table ... <CONCEPT>
Rows total:     444

Processing table ... <DRUG_EXPOSURE>
Rows processed: 1000
Rows processed: 2000
Rows processed: 3000
Rows processed: 4000
Rows processed: 5000
Rows processed: 6000
Rows processed: 7000
Rows processed: 8000
Rows processed: 9000
Rows processed: 10000
Rows processed: 11000
Rows processed: 12000
Rows processed: 13000
Rows processed: 14000
Rows processed: 15000
Rows processed: 16000
Rows processed: 17000
Rows processed: 18000
Rows processed: 19000
Rows processed: 20000
Rows processed: 21000
Rows processed: 22000
Rows pr

### Stream mode

In the default mode `--no-stream`, all the data are loaded into RAM and then consolidated into a single JSON file.

However, if your OMOP CDM is large, this can be problematic because the required amount of RAM may exceed your hardware specifications. In such cases, we recommend using `--stream`, which enables processing your OMOP CDM input data line by line.

Note, however, that `CONCEPT`, `PERSON` and `VISIT_OCCURRENCE` tables have to be loaded in RAM memory.

 First we will perform a basic conversion. This one will create an error on purpose, due to the fact that the `CONCEPT` table is also read line-by-line and the default value of `-max-lines-sql 500` does not capture `concept_id` outside this range.

In [ ]:
!bin/convert-pheno -iomop t/omop2bff/in/omop_cdm_eunomia.sql -obff omop_individuals_stream.json --out-dir /content -O -verbose -stream

****************************************
*  Phenotypic Data Model Convert Tool  *
*          - CONVERT-PHENO -           *
*            Version: 0.20_1             *
*   (C) 2022-2024 Manuel Rueda, CNAG   *
*       The Artistic License 2.0       *
****************************************

==== METHOD: <omop2bff> ====
Processing: <t/omop2bff/in/omop_cdm_eunomia.sql>
Writing </content/omop_individuals_stream.json> file

Processing table ... <CONCEPT>
Rows total:     444

Processing table ... <PERSON>
Rows total:     500

Processing table ... <VISIT_OCCURRENCE>
Rows total:     500

Processing table ... <CONDITION_OCCURRENCE>
Error: We could not find person_id:608. Try increasing the #lines with --max-lines-sql


Thus, we add the option `-max-lines-sql 99999` and re-run:

In [ ]:
!bin/convert-pheno -iomop t/omop2bff/in/omop_cdm_eunomia.sql -obff omop_individuals_stream.json --out-dir /content -O -verbose -stream -max-lines-sql 99999

****************************************
*  Phenotypic Data Model Convert Tool  *
*          - CONVERT-PHENO -           *
*            Version: 0.20_1             *
*   (C) 2022-2024 Manuel Rueda, CNAG   *
*       The Artistic License 2.0       *
****************************************

==== METHOD: <omop2bff> ====
Processing: <t/omop2bff/in/omop_cdm_eunomia.sql>
Writing </content/omop_individuals_stream.json> file

Processing table ... <CONCEPT>
Rows total:     444

Processing table ... <PERSON>
Rows processed: 1000
Rows processed: 2000
Rows total:     2694

Processing table ... <VISIT_OCCURRENCE>
Rows processed: 1000
Rows total:     1037

Processing table ... <CONDITION_OCCURRENCE>
Rows processed: 10000
Rows processed: 20000
Rows processed: 30000
Rows processed: 40000
Rows processed: 50000
Rows processed: 60000
Rows total:     65332

Processing table ... <PROCEDURE_OCCURRENCE>
Rows processed: 10000
Rows processed: 20000
Rows processed: 30000
Rows total:     37409

Processing table 

Ok, let's take a look to the data:

In [ ]:
!head -5 /content/omop_individuals_stream.json

{"diseases":[{"_info":{"CONDITION_OCCURRENCE":{"OMOP_columns":{"condition_concept_id":4112343,"condition_end_date":"2015-10-14","condition_end_datetime":"2015-10-14 00:00:00","condition_occurrence_id":4483,"condition_source_concept_id":4112343,"condition_source_value":195662009,"condition_start_date":"2015-10-02","condition_start_datetime":"2015-10-02 00:00:00","condition_status_concept_id":0,"condition_status_source_value":null,"condition_type_concept_id":32020,"person_id":"263","provider_id":"\\N","stop_reason":null,"visit_detail_id":0,"visit_occurrence_id":17479}}},"ageOfOnset":{"age":{"iso8601duration":"61Y"}},"diseaseCode":{"id":"SNOMED:195662009","label":"Acute viral pharyngitis"},"stage":{"id":"None:No matching concept","label":"No matching concept"}}],"ethnicity":{"id":"NCIT:C41261","label":"White"},"geographicOrigin":{"id":"NCIT:C43853","label":"English"},"id":"263","info":{"PERSON":{"OMOP_columns":{"birth_datetime":"1954-06-20 00:00:00","care_site_id":"\\N","day_of_birth":20,

This time we have one JSON object per line, and the data are not consolidated per individual. If your goal is to ingest these data into `MongoDB`, this is a cosmetic concern as the joining will be carried out by the field `id`.

Anything said for the `--no-stream` mode works here. Let's do a conversion using only one table.


In [ ]:
!bin/convert-pheno -iomop t/omop2bff/in/omop_cdm_eunomia.sql -obff omop_individuals_drug_exposure_stream.json --omop-tables DRUG_EXPOSURE -O --out-dir /content --verbose -max-lines-sql 99999 --stream

****************************************
*  Phenotypic Data Model Convert Tool  *
*          - CONVERT-PHENO -           *
*            Version: 0.20_1             *
*   (C) 2022-2024 Manuel Rueda, CNAG   *
*       The Artistic License 2.0       *
****************************************

==== METHOD: <omop2bff> ====
Processing: <t/omop2bff/in/omop_cdm_eunomia.sql>
Writing </content/omop_individuals_drug_exposure_stream.json> file

Processing table ... <CONCEPT>
Rows total:     444

Processing table ... <PERSON>
Rows processed: 1000
Rows processed: 2000
Rows total:     2694

Processing table ... <VISIT_OCCURRENCE>
Rows processed: 1000
Rows total:     1037

Processing table ... <DRUG_EXPOSURE>
Rows processed: 10000
Rows processed: 20000
Rows processed: 30000
Rows processed: 40000
Rows processed: 50000
Rows processed: 60000
Rows total:     67707



# Analyzing the efficiency of the conversion

We are going to use some scripts to analyze the efficiency of the conversion. We will start by counting the number of entries of each CSV.

First, we will run `Convert-Pheno`making sure that it includes all PosgreSQL rows in the conversion:

In [ ]:
!bin/convert-pheno -iomop t/omop2bff/in/omop_cdm_eunomia.sql -obff omop_individuals_efficiency.json --out-dir /content --sql2csv --verbose -O -max-lines-sql 99999

****************************************
*  Phenotypic Data Model Convert Tool  *
*          - CONVERT-PHENO -           *
*            Version: 0.20_1             *
*   (C) 2022-2024 Manuel Rueda, CNAG   *
*       The Artistic License 2.0       *
****************************************

==== METHOD: <omop2bff> ====
Processing: <t/omop2bff/in/omop_cdm_eunomia.sql>
Writing </content/omop_individuals_efficiency.json> file

Processing table ... <CONCEPT>
Rows total:     444

Processing table ... <CONDITION_OCCURRENCE>
Rows processed: 1000
Rows processed: 2000
Rows processed: 3000
Rows processed: 4000
Rows processed: 5000
Rows processed: 6000
Rows processed: 7000
Rows processed: 8000
Rows processed: 9000
Rows processed: 10000
Rows processed: 11000
Rows processed: 12000
Rows processed: 13000
Rows processed: 14000
Rows processed: 15000
Rows processed: 16000
Rows processed: 17000
Rows processed: 18000
Rows processed: 19000
Rows processed: 20000
Rows processed: 21000
Rows processed: 22000
Row

Let's do a rough count using `Bash` commands:

In [ ]:
!wc -l /content/*csv

     445 /content/CONCEPT.csv
   65333 /content/CONDITION_OCCURRENCE.csv
   67708 /content/DRUG_EXPOSURE.csv
   44054 /content/MEASUREMENT.csv
    1478 /content/OBSERVATION.csv
    2695 /content/PERSON.csv
   37410 /content/PROCEDURE_OCCURRENCE.csv
    1038 /content/VISIT_OCCURRENCE.csv
  220161 total


Well, that worked but the stats are a bit simple...
Note that each CSV has a header so the total number of entries is n-1.

Let's continue using `Bash` to perform a more sophisticated processing. In the cells below, first we create on-the-fly a the script named `count_lines.sh` and then we run it.

In [ ]:
# @title See code
%%bash
# Define the script
cat << 'EOF' > count_lines.sh
#!/bin/bash

# Check if at least one file is provided
if [ "$#" -eq 0 ]; then
    echo "Usage: $0 file1 file2 ..."
    exit 1
fi

# Loop through each provided file
for file in "$@"; do
    # Get the basename of the file without the extension
    base=$(basename "$file" | cut -d. -f1)

    # Count the number of lines excluding the header
    line_count=$(sed 1d "$file" | wc -l)

    # Output the basename and line count separated by a tab
    echo -e "${base}\t${line_count}"
done
EOF

# Make the script executable
chmod +x count_lines.sh

In [ ]:
!./count_lines.sh /content/*csv > /content/csv_stats.txt
!cat /content/csv_stats.txt

CONCEPT	444
CONDITION_OCCURRENCE	65332
DRUG_EXPOSURE	67707
MEASUREMENT	44053
OBSERVATION	1477
PERSON	2694
PROCEDURE_OCCURRENCE	37409
VISIT_OCCURRENCE	1037


Great, now let's count how many of these rows were converted into BFF. Again, we will create a script on the fly `process_json.sh` and then we execute it.

In [ ]:
# @title See code
%%bash
# Define the script
cat << 'EOF' > process_json.sh
#!/bin/bash

# Check if input file is provided
if [ "$#" -ne 1 ]; then
    echo "Usage: $0 input_file"
    exit 1
fi

input=$1

# Loop through the terms and process the JSON input
for term in diseases exposures id interventionsOrProcedures measures phenotypicFeatures treatments
do
  echo -en "$term\t"
  if [ "$term" = "id" ]; then
    # Count 'id' only at the top level
    jq -c '[.[] | select(has("id"))] | length' < $input
  else
    # Checking for presence of the term and if it's an array; count elements
    jq -c '[.[] | .["'"$term"'"]? | if type == "array" then . | length else 0 end] | add' < $input
  fi
done
EOF

# Make the script executable
chmod +x process_json.sh

In [ ]:
!./process_json.sh /content/omop_individuals_efficiency.json > /content/json_stats.txt
!cat /content/json_stats.txt

diseases	65332
exposures	0
id	2694
interventionsOrProcedures	37409
measures	0
phenotypicFeatures	1477
treatments	67707


Ok, now let's create a table like that of the paper. We'll be using a `Python`script:

In [ ]:
# @title See code
# Define the script with an output mode switch
script_content = """
import sys
from IPython.display import Markdown

# Ensure the correct number of arguments are provided
if len(sys.argv) != 4:
    print("Usage: python script.py <json> <csv> <mode>")
    sys.exit(1)

# Read contents from the files specified in the arguments
file1_path = sys.argv[1]
file2_path = sys.argv[2]
mode = sys.argv[3]  # 'raw' or 'display'

with open(file1_path, 'r') as file:
    file1_content = file.read()

with open(file2_path, 'r') as file:
    file2_content = file.read()

# Read and parse file1
bff_dict = {}
for line in file1_content.strip().split('\\n'):
    key, value = line.split('\\t')
    bff_dict[key] = int(value)

# Read and parse file2
csv_dict = {}
for line in file2_content.strip().split('\\n'):
    key, value = line.split('\\t')
    csv_dict[key] = int(value)

# Define the mapping from OMOP-CDM table names to BFF term names
mapping = {
    "CONDITION_OCCURRENCE": "diseases",
    "DRUG_EXPOSure": "treatments",
    "MEASUREMENT": "measures",
    "OBSERVATION": "phenotypicFeatures",
    "PERSON": "id",
    "PROCEDURE_OCCURRENCE": "interventionsOrProcedures",
    "CONCEPT": None,
    "VISIT_OCCURRENCE": None
}

# Generate the markdown table
markdown_table = "| OMOP-CDM Table          | BFF Term                     | # Rows in CSV | # Objects in BFF | Efficiency (%) |\\n"
markdown_table += "|-------------------------|------------------------------|---------------|------------------|----------------|\\n"

for table, csv_rows in csv_dict.items():
    bff_term = mapping.get(table, "-")
    bff_rows = bff_dict.get(bff_term, 0) if bff_term else "-"
    efficiency = f"{((bff_rows / csv_rows) * 100):.0f}" if isinstance(bff_rows, int) and bff_rows > 0 else "-"
    markdown_table += f"| {table.ljust(24)} | {bff_term if bff_term else '-'.ljust(28)} | {csv_rows:15,} | {bff_rows:17} | {efficiency:15} |\\n"

# Display the table based on the mode
if mode == 'display':
    Markdown(markdown_table)
else:
    print(markdown_table)
"""

# Write the script content to a Python file
with open('create_table.py', 'w') as file:
    file.write(script_content)

Now execute the script:

In [ ]:
!python3 create_table.py /content/json_stats.txt /content/csv_stats.txt display

| OMOP-CDM Table          | BFF Term                     | # Rows in CSV | # Objects in BFF | Efficiency (%) |
|-------------------------|------------------------------|---------------|------------------|----------------|
| CONCEPT                 | -                            | 444           | -                | -              |
| CONDITION_OCCURRENCE    | diseases                     | 65,332        | 65,332           | 100            |
| DRUG_EXPOSURE           | treatments                   | 67,707        | 67,707           | 100            |
| MEASUREMENT             | measures                     | 44,053**      | 0                | 100            |
| OBSERVATION             | phenotypic-Features          | 1,477         | 1,477            | 100            |
| PERSON                  | id                           | 2,694         | 2,694            | 100            |
| PROCEDURE_OCCURRENCE    | Interventions-Or-Procedures  | 37,409        | 37,409           | 100            |
| VISIT_OCCURRENCE        | -                            | 1,037         | -                | -              |                                           

Nice! Now let's do the same with a file extracted with `--stream` option.

First we run `Convert-Pheno` in **stream** mode:

In [ ]:
!bin/convert-pheno -iomop t/omop2bff/in/omop_cdm_eunomia.sql -obff omop_individuals_efficiency_stream.json --out-dir /content --verbose -O -max-lines-sql 99999 --stream

****************************************
*  Phenotypic Data Model Convert Tool  *
*          - CONVERT-PHENO -           *
*            Version: 0.20_1             *
*   (C) 2022-2024 Manuel Rueda, CNAG   *
*       The Artistic License 2.0       *
****************************************

==== METHOD: <omop2bff> ====
Processing: <t/omop2bff/in/omop_cdm_eunomia.sql>
Writing </content/omop_individuals_efficiency_stream.json> file

Processing table ... <CONCEPT>
Rows total:     444

Processing table ... <PERSON>
Rows processed: 1000
Rows processed: 2000
Rows total:     2694

Processing table ... <VISIT_OCCURRENCE>
Rows processed: 1000
Rows total:     1037

Processing table ... <CONDITION_OCCURRENCE>
Rows processed: 10000
Rows processed: 20000
Rows processed: 30000
Rows processed: 40000
Rows processed: 50000
Rows processed: 60000
Rows total:     65332

Processing table ... <PROCEDURE_OCCURRENCE>
Rows processed: 10000
Rows processed: 20000
Rows processed: 30000
Rows total:     37409

Proces

Let's run the script to get the stats:

In [ ]:
!head -2 /content/omop_individuals_efficiency_stream.json | cut -c1-100

{"diseases":[{"_info":{"CONDITION_OCCURRENCE":{"OMOP_columns":{"condition_concept_id":4112343,"condi
{"diseases":[{"_info":{"CONDITION_OCCURRENCE":{"OMOP_columns":{"condition_concept_id":192671,"condit


We need to create a new script to parse this new data:

In [ ]:
# @title See code
%%bash
# Define the script
cat << 'EOF' > process_json_stream.py
import sys
import json

if len(sys.argv) != 2:
    print("Usage: python script.py <input_file>")
    sys.exit(1)

input_file = sys.argv[1]

# Initialize counts
counts = {
    "diseases": 0,
    "exposures": 0,
    "interventionsOrProcedures": 0,
    "measures": 0,
    "phenotypicFeatures": 0,
    "treatments": 0
}

unique_ids = set()

# Process each JSON object line-by-line
with open(input_file, 'r') as f:
    for line in f:
        line = line.strip()
        if not line:
            continue  # Skip empty lines
        try:
            data = json.loads(line)
        except json.JSONDecodeError:
            print(f"Skipping invalid JSON line: {line}")
            continue  # Skip lines that are not valid JSON

        for term in counts.keys():
            if term in data and isinstance(data[term], list):
                counts[term] += len(data[term])

        # Handle unique ids
        if "id" in data and data["id"] is not None:
            unique_ids.add(data["id"])

# Output results with tab-separated values
for term, count in counts.items():
    print(f"{term}\t{count}")

# Output unique id count
print(f"id\t{len(unique_ids)}")
EOF

In [ ]:
!python3 ./process_json_stream.py /content/omop_individuals_efficiency_stream.json > /content/json_stats_stream.txt
!cat /content/json_stats_stream.txt

diseases	65332
exposures	0
interventionsOrProcedures	37409
measures	0
phenotypicFeatures	1477
treatments	67707
id	2694


Now we build the efficiency table again:

In [ ]:
!python3 create_table.py /content/json_stats_stream.txt /content/csv_stats.txt raw

| OMOP-CDM Table          | BFF Term                     | # Rows in CSV | # Objects in BFF | Efficiency (%) |
|-------------------------|------------------------------|---------------|------------------|----------------|
| CONCEPT                  | -                            |             444 | -                 | -               |
| CONDITION_OCCURRENCE     | diseases |          65,332 |             65332 | 100             |
| DRUG_EXPOSURE            | - |          67,707 |                 0 | -               |
| MEASUREMENT              | measures |          44,053 |                 0 | -               |
| OBSERVATION              | phenotypicFeatures |           1,477 |              1477 | 100             |
| PERSON                   | id |           2,694 |              2694 | 100             |
| PROCEDURE_OCCURRENCE     | interventionsOrProcedures |          37,409 |             37409 | 100             |
| VISIT_OCCURRENCE         | -                            |           1,

# Optional

## Downloading the Athena-OHDSI database

If your OMOP CDM data is not self-contained, you may need an external database to fetch the `concept_id`. To facilitate this, we will download the provided Athena-OHDSI database.

Unfortunately, Google does not make it easy to use `wget, curl, etc.` to perform the download so we will install Python's library `gdown`:

In [ ]:
!pip install gdown

And then download the actual file to /content/ohdsi.db

In [ ]:
# @title Show code
import gdown

url = 'https://drive.google.com/uc?export=download&id=1-Ls1nmgxp-iW-8LkRIuNNdNytXa8kgNw'
output = '/content/ohdsi.db'
gdown.download(url, output, quiet=False)

Downloading...
From (original): https://drive.google.com/uc?export=download&id=1-Ls1nmgxp-iW-8LkRIuNNdNytXa8kgNw
From (redirected): https://drive.google.com/uc?export=download&id=1-Ls1nmgxp-iW-8LkRIuNNdNytXa8kgNw&confirm=t&uuid=1a85a21f-706b-481a-bf4d-5ddd23c5d064
To: /content/ohdsi.db
100%|██████████| 2.30G/2.30G [00:29<00:00, 77.1MB/s]


'/content/ohdsi.db'

## Uploading files

It's possible to run jobs using your own files. A few notes:

* The files will be stored in **your instance** of this document. We (developers) don't have access to them.

* Uploaded files will get **deleted** when this runtime is recycled.

* `Convert-Pheno` does not send remote information to our servers and does not store user's information.

* The security is dictated by your Google account / drive.

The files can be uploaded by clicking in the upload button on the left tab of this screen. Files will be uploaded to `/content`.

Once the file is uploaded, you can execute `convert-pheno` command-line as we did above.

Great! I hope this quick tour was useful.

Now it's time to try it out yourself!

Bye!!

Manu